In [1]:
%%writefile mapper_total.py
#!/usr/bin/env python2
import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print "%d" % (1)

Writing mapper_total.py


In [2]:
%%writefile reducer_total.py
#!/usr/bin/env python2
import sys

total = 0

for line in sys.stdin:
    total += 1

print "%s\t%d" % ('total', total)

Writing reducer_total.py


In [3]:
%%writefile mapper_find_names.py
#!/usr/bin/env python2
import sys
import re

def isWordName(word):
    if len(word) == 0 or word[0].isdigit() or (not word[0].isupper()):
        return False
    else:
        if len(word) > 1:
            for c in word[1:]:
                if c.isupper():
                    return False
        return True

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        if isWordName(word):
            print "%s\t%d" % (word.lower(), 1)

Writing mapper_find_names.py


In [15]:
%%writefile reducer_find_names.py
#!/usr/bin/env python2
import sys

current_key = None
word_sum = 0
total = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Overwriting reducer_find_names.py


In [10]:
%%writefile reducer_freq.py
#!/usr/bin/env python2
import sys

current_key = None
word_sum = 0
total = 0

with open('paart-00000') as f:
    tmp = f.readline()
total = int(tmp.strip().split('\t', 1)[1])

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            freq = (word_sum / (total * 1.0)) * 100
            if freq < 0.5:
                print "%s\t%d\t%f" % (current_key, word_sum, freq)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    freq = (word_sum / (total * 1.0)) * 100
    if freq < 0.5:
        print "%s\t%d\t%f" % (current_key, word_sum, freq)

Writing reducer_freq.py


In [11]:
! chmod +x ./reducer_find_names.py

In [8]:
! hdfs dfs -copyToLocal /data/wiki/en_articles_part/articles-part

In [18]:
! cat articles-part | ./mapper_find_names.py | sort | ./reducer_find_names.py > my_file.txt

In [16]:
! ls -l

total 75124
-rw-r--r-- 1 jovyan users 76861985 Nov 23 05:16 articles-part
-rw-rw-r-- 1 jovyan root      1634 Oct  9 00:28 Demo.ipynb
-rwxr-xr-x 1 jovyan users      687 Nov 23 05:09 mapper_find_names.py
-rw-r--r-- 1 jovyan users      369 Nov 23 05:03 mapper_total.py
-rw-r--r-- 1 jovyan users        0 Nov 23 05:19 my_file.txt
-rw-r--r-- 1 jovyan users    22572 Nov 23 05:19 name_rating.ipynb
-rw-r--r-- 1 jovyan users       15 Nov 23 05:15 part-00000
-rw-rw-r-- 1 jovyan root       239 Oct 15 16:10 README.md
-rwxr-xr-x 1 jovyan users      465 Nov 23 05:20 reducer_find_names.py
-rw-r--r-- 1 jovyan users      118 Nov 23 05:03 reducer_total.py
-rw-r--r-- 1 root   users      757 Nov 23 05:01 supervisord.log
-rw-r--r-- 1 root   users        2 Nov 23 05:01 supervisord.pid


In [23]:
! cat freq_part-00000 | sort -k2nr | cat > my_output.txt

In [5]:
! cat freq_part-00000 | ./mapper_freq.py >> my_output.txt

# Job runner

In [19]:
%%bash
TOTAL_DIR="total"
NAMES_DIR="names"
#FREQ_DIR="freq"
#OUT_DIR="wordcount_stopwords_result"

#rm part-00000
hdfs dfs -rm -r -skipTrash ${TOTAL_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${NAMES_DIR} > /dev/null
#hdfs dfs -rm -r -skipTrash ${FREQ_DIR} > /dev/null
#hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Total_count" \
    -D mapreduce.job.reduces=1 \
    -files mapper_total.py,reducer_total.py \
    -mapper "python mapper_total.py" \
    -reducer "python reducer_total.py" \
    -input /data/wiki/en_articles_part \
    -output ${TOTAL_DIR} > /dev/null

hdfs dfs -copyToLocal ./total/part-00000 .

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Names_count" \
    -D mapreduce.job.reduces=1 \
    -files mapper_find_names.py,reducer_find_names.py \
    -mapper "python mapper_find_names.py" \
    -reducer "python reducer_find_names.py" \
    -input /data/wiki/en_articles_part \
    -output ${NAMES_DIR} > /dev/null

17/11/23 05:28:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/23 05:28:24 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/23 05:28:25 INFO mapred.FileInputFormat: Total input files to process : 1
17/11/23 05:28:25 INFO mapreduce.JobSubmitter: number of splits:2
17/11/23 05:28:25 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
17/11/23 05:28:26 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1511413328306_0003
17/11/23 05:28:26 INFO impl.YarnClientImpl: Submitted application application_1511413328306_0003
17/11/23 05:28:26 INFO mapreduce.Job: The url to track the job: http://190df3c96206:8088/proxy/application_1511413328306_0003/
17/11/23 05:28:26 INFO mapreduce.Job: Running job: job_1511413328306_0003
17/11/23 05:28:32 INFO mapreduce.Job: Job job_1511413328306_0003 running in uber mode : false
17/11/23 05:28:32 INFO mapreduce.Job:  map 0% reduce 0%
17/11/23 05:28:49 INFO m

In [ ]:
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Frequencies_of_names" \
    -D mapreduce.job.reduces=1 \
    -files mapper_freq.py,reducer_freq.py,part-00000 \
    -mapper "python mapper_freq.py" \
    -reducer "python reducer_freq.py" \
    -input /data/wiki/en_articles_part \
    -output ${FREQ_DIR} > /dev/null    
    
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Frequencies_of_names" \
    -D mapreduce.job.reduces=1 \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2nr" \
    -files mapper_freq.py,reducer_freq.py,part-00000 \
    -mapper "cat" \
    -reducer "cat" \
    -input ${FREQ_DIR}/part-* \
    -output ${OUT_DIR} > /dev/null        

hdfs dfs -cat ${FREQ_DIR}/part-0000* | head  

In [30]:
!hdfs dfs -copyToLocal ./total/part-00000 .

copyToLocal: `part-00000': File exists


In [16]:
!hdfs dfs -copyToLocal freq/part-00000 freq_part-00000

In [17]:
! ls -l

total 120456
-rw-r--r-- 1 jovyan users 76861985 Nov 14 05:16 articles-part
-rw-rw-r-- 1 jovyan root      1634 Oct  9 00:28 Demo.ipynb
-rw-r--r-- 1 jovyan users  3577563 Nov 14 05:41 freq_part-00000
-rwxr-xr-x 1 jovyan users      687 Nov 14 05:21 mapper_freq.py
-rw-r--r-- 1 jovyan users      369 Nov 14 05:21 mapper_total.py
-rw-r--r-- 1 jovyan users 42835716 Nov 14 05:18 my_output.txt
-rw-r--r-- 1 jovyan users    20928 Nov 14 05:41 name_rating (1).ipynb
-rw-r--r-- 1 jovyan users       15 Nov 14 05:36 part-00000
-rw-rw-r-- 1 jovyan root       239 Oct 15 16:10 README.md
-rw-r--r-- 1 jovyan users      732 Nov 14 05:21 reducer_freq.py
-rw-r--r-- 1 jovyan users      118 Nov 14 05:21 reducer_total.py
-rw-r--r-- 1 root   users      757 Nov 14 05:03 supervisord.log
-rw-r--r-- 1 root   users        2 Nov 14 05:03 supervisord.pid
